In [1]:
from stellargraph.data import EdgeSplitter
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.decomposition import PCA
import stellargraph as sg
import matplotlib.pyplot as plt
import networkx as nx
import common_functions as cf
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import classification_report
import random
import stellargraph as sg
from collections import Counter

In [2]:
G, mapper, reverse_mapper = cf.create_mapping_from_file_path('facebook/0.edges')

edge_splitter = EdgeSplitter(G)
splitted_graph, X, y = edge_splitter.train_test_split(p=0.2, method="global")

** Sampled 503 positive and 503 negative edges. **


# Baseline Recommender
This rule-based algorithm finds the top 5 most common friends for each node.
 
We will use this metric to recommend the top 5 most frequent friends. If there's not enough to recommend, we randomly select nodes.

In [3]:
def top_common_neighbors(graph):
    # Get all node ids in the graph
    all_node_ids = list(graph.nodes())

    # Initialize an empty dictionary to store the results
    result = {}

    # Iterate over each node in the graph
    for node in all_node_ids:
        # Get the neighbors of the node
        neighbors = list(graph.neighbors(node))

        # Count the number of common neighbors for each neighbor
        common_neighbors_counts = Counter([
            neighbor for n in neighbors for neighbor in graph.neighbors(n) if neighbor != node
        ])

        # Get the top 5 nodes with the most common neighbors
        top_5 = [node_id for node_id, count in common_neighbors_counts.most_common(5)]

        # If the node has less than 5 neighbors, fill the remaining spots with random node ids
        while len(top_5) < 5:
            random_node = random.choice(all_node_ids)
            if random_node != node and random_node not in top_5:
                top_5.append(random_node)

        # Add the result to the dictionary
        result[node] = top_5

    return result

In [4]:
neighbor_dict = top_common_neighbors(splitted_graph)

In [5]:
for i in range(2):
    print(f'{list(neighbor_dict.keys())[i]}: {neighbor_dict[i]}')

0: [36, 85, 15, 1, 34]
56: [60, 36, 34, 109, 15]


In [6]:
predictions = []
for (x1, x2), y_ in zip(X,y):
    if y_ == 1:
        if x1 in neighbor_dict[x2] or x2 in neighbor_dict[x1]:
            predictions.append(1)
        else:
            predictions.append(0)
    else:
        if x1 in neighbor_dict[x2] or x2 in neighbor_dict[x1]:
            predictions.append(0)
        else:
            predictions.append(1)

In [7]:
predictions = np.array(predictions)

# Evaluation

In [8]:
accuracy = accuracy_score(y, predictions)*100
print(f'Accuracy: {accuracy:.2f}%')

Accuracy: 18.89%


In [9]:
cm = confusion_matrix(y, predictions)
cm

array([[  8, 495],
       [321, 182]], dtype=int64)

In [10]:
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       0.02      0.02      0.02       503
           1       0.27      0.36      0.31       503

    accuracy                           0.19      1006
   macro avg       0.15      0.19      0.16      1006
weighted avg       0.15      0.19      0.16      1006
